## Libraries

In [1]:
try:
    import google.colab
    from google.colab import drive
    drive.mount('/content/drive')
    IN_COLAB = True
except:
    IN_COLAB = False

Mounted at /content/drive


In [2]:
if IN_COLAB:
    !pip install transformers
    !pip install datasets
    !pip install evaluate
    !pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 41.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 59.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 13.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 7.9 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-whe

In [3]:
import os
import torch

if IN_COLAB:
    root_path = '/content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA'
else:
    root_path = '/Users/kscaria/Library/CloudStorage/GoogleDrive-scariakevin1@gmail.com/My Drive/Knowledge/MSIT/Research/InstructABSA'
    
use_mps = True if torch.has_mps else False
os.chdir(root_path)

In [4]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd

from InstructABSA.utils import T5ATE
from InstructABSA.data_prep import ModelReadyData

In [5]:
rest_train_file_path = './Data/semeval14/ABSA_TrainData/Restaurants_Train_v2.csv'
laptops_train_file_path = './Data/semeval14/ABSA_TrainData/Laptop_Train_v2.csv'
rest_test_file_path = './Data/semeval14/ABSA_Gold_TestData/Restaurants_Test_Gold.csv'
laptops_test_file_path = './Data/semeval14/ABSA_Gold_TestData/Laptops_Test_Gold.csv'

# Load the data
res_tr_df = pd.read_csv(rest_train_file_path)
lap_tr_df = pd.read_csv(laptops_train_file_path)
res_te_df = pd.read_csv(rest_test_file_path)
lap_te_df = pd.read_csv(laptops_test_file_path)

In [6]:
# Extract the aspect term and polarity for each review
model_data = ModelReadyData()

# Get the input text into the required format
bos_instruction = 'Review: '
eos_instruction = ' Aspect: '
res_tr_df = model_data.create_data_in_ate_format(res_tr_df, 'term', 'raw_text', 'aspectTerms', 'labels', bos_instruction, eos_instruction)
lap_tr_df = model_data.create_data_in_ate_format(lap_tr_df, 'term', 'raw_text', 'aspectTerms', 'labels', bos_instruction, eos_instruction)
res_te_df = model_data.create_data_in_ate_format(res_te_df, 'term', 'raw_text', 'aspectTerms', 'labels', bos_instruction, eos_instruction)
lap_te_df = model_data.create_data_in_ate_format(lap_te_df, 'term', 'raw_text', 'aspectTerms', 'labels', bos_instruction, eos_instruction)

In [29]:
lap_tr_df[lap_tr_df['labels'].apply(lambda x: len(x.split(', '))==13)].loc[655, 'labels']

'programs, Garageband, Photo Booth, iPhoto, iMovie, video-editing, movie-making, photo management, music, Pages, Numbers, Keynote, school or office use'

In [14]:
res_tr_df[res_tr_df['labels']=='noaspectterm']

,sentenceId,raw_text,aspectTerms,aspectCategories,labels,text
3,2534,Where Gabriela personaly greets you and recomm...,"[{'term': 'noaspectterm', 'polarity': 'none'}]","[{'category': 'service', 'polarity': 'positive'}]",noaspectterm,Review: Where Gabriela personaly greets you an...
4,583,"For those that go once and don't enjoy it, all...","[{'term': 'noaspectterm', 'polarity': 'none'}]","[{'category': 'anecdotes/miscellaneous', 'pola...",noaspectterm,Review: For those that go once and don't enjoy...
6,1571,It is very overpriced and not very tasty.,"[{'term': 'noaspectterm', 'polarity': 'none'}]","[{'category': 'food', 'polarity': 'negative'},...",noaspectterm,Review: It is very overpriced and not very tas...
14,3119,I would definitely recommend Mary's and am mak...,"[{'term': 'noaspectterm', 'polarity': 'none'}]","[{'category': 'anecdotes/miscellaneous', 'pola...",noaspectterm,Review: I would definitely recommend Mary's an...
16,70,My Girlfriend and I stumbled onto this hopping...,"[{'term': 'noaspectterm', 'polarity': 'none'}]","[{'category': 'anecdotes/miscellaneous', 'pola...",noaspectterm,Review: My Girlfriend and I stumbled onto this...
...,...,...,...,...,...,...
3031,3168,"Better than the bagel shop on the corner, but ...","[{'term': 'noaspectterm', 'polarity': 'none'}]","[{'category': 'anecdotes/miscellaneous', 'pola...",noaspectterm,Review: Better than the bagel shop on the corn...
3032,2798,it helps if you know what to order.,"[{'term': 'noaspectterm', 'polarity': 'none'}]","[{'category': 'anecdotes/miscellaneous', 'pola...",noaspectterm,Review: it helps if you know what to order. As...
3036,1063,But that is highly forgivable.,"[{'term': 'noaspectterm', 'polarity': 'none'}]","[{'category': 'anecdotes/miscellaneous', 'pola...",noaspectterm,Review: But that is highly forgivable. Aspect:
3038,875,"When we arrived at 6:00 PM, the restaurant was...","[{'term': 'noaspectterm', 'polarity': 'none'}]","[{'category': 'anecdotes/miscellaneous', 'pola...",noaspectterm,"Review: When we arrived at 6:00 PM, the restau..."


In [17]:
2042-1020

1022

In [15]:
res_tr_df['labels'].apply(lambda x: len(x.split(', '))).value_counts()

1    2042
2     572
3     269
4     104
5      30
6      15
7       5
8       3
9       1
Name: labels, dtype: int64

In [ ]:
# Experimentation
experiment_id = 'laptops'
experiment_name = 'laptops_vanilla'
model_checkpoint = 't5-base'

# Create T5 utils object
t5_exp = T5ATE(model_checkpoint, experiment_id, res_tr_df, res_te_df, lap_tr_df, lap_te_df)

if IN_COLAB:
    model_out_path = os.path.join(root_path, 'T5', 'ATE')
else:
    model_out_path = os.getcwd()

model_out_path = os.path.join(model_out_path, f"{model_checkpoint}-{experiment_name}", "checkpoints")
print('Model output path: ', model_out_path)


# Tokenize Datasets
id_dataset, ood_dataset, id_tokenized_dataset, ood_tokenized_dataset = t5_exp.set_data_for_training_semeval(experiment_id)

# Training arguments
training_args = {
    'output_dir':model_out_path,
    'evaluation_strategy':"epoch",
    'learning_rate':5e-5,
    'per_device_train_batch_size':16,
    'per_device_eval_batch_size':16,
    'num_train_epochs':4,
    'weight_decay':0.01,
    'warmup_ratio':0.1,
    'save_strategy':'no',
    'load_best_model_at_end':False,
    'push_to_hub':False,
    'eval_accumulation_steps':1,
    'predict_with_generate':True,
    'use_mps_device':use_mps
}

Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

Model output path:  /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATE/t5-base-laptops_vanilla/checkpoints


  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
# Train Model
model_trainer = t5_exp.train(id_tokenized_dataset, **training_args)

***** Running training *****
  Num examples = 3045
  Num Epochs = 4
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 764
  Number of trainable parameters = 222903552
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Trainer device: cuda:0

Model training started ....


Epoch,Training Loss,Validation Loss
1,No log,0.277487
2,No log,0.202690
3,1.005600,0.196367
4,1.005600,0.191943


***** Running Evaluation *****
  Num examples = 800
  Batch size = 16
***** Running Evaluation *****
  Num examples = 800
  Batch size = 16
***** Running Evaluation *****
  Num examples = 800
  Batch size = 16
***** Running Evaluation *****
  Num examples = 800
  Batch size = 16


Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATE/t5-base-laptops_vanilla/checkpoints
Configuration saved in /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATE/t5-base-laptops_vanilla/checkpoints/config.json
Model weights saved in /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATE/t5-base-laptops_vanilla/checkpoints/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATE/t5-base-laptops_vanilla/checkpoints/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/Knowledge/MSIT

In [ ]:
# Model inference
best_model = 'checkpoints'
print('Getting model from path: ', model_out_path)

# Get prediction labels - Training set
id_tr_pred_labels = t5_exp.get_labels(predictor = model_trainer, tokenized_dataset = id_tokenized_dataset, sample_set = 'train')

# Get prediction labels - Testing set
id_te_pred_labels = t5_exp.get_labels(predictor = model_trainer, tokenized_dataset = id_tokenized_dataset, sample_set = 'validation')

# Get prediction labels - OOD Training set
ood_tr_pred_labels = t5_exp.get_labels(predictor = model_trainer, tokenized_dataset = ood_tokenized_dataset, sample_set = 'train')

# Get prediction labels - OOD Training set
ood_te_pred_labels = t5_exp.get_labels(predictor = model_trainer, tokenized_dataset = ood_tokenized_dataset, sample_set = 'validation')

***** Running Prediction *****
  Num examples = 3045
  Batch size = 16


Getting model from path:  /content/drive/MyDrive/Knowledge/MSIT/Research/InstructABSA/T5/ATE/t5-base-laptops_vanilla/checkpoints


***** Running Prediction *****
  Num examples = 800
  Batch size = 16


***** Running Prediction *****
  Num examples = 3041
  Batch size = 16


***** Running Prediction *****
  Num examples = 800
  Batch size = 16


In [ ]:
# Add new column in the respective dataframes
id_tr_df = pd.DataFrame(id_dataset['train'])
id_te_df = pd.DataFrame(id_dataset['validation'])
ood_tr_df = pd.DataFrame(ood_dataset['train'])
ood_te_df = pd.DataFrame(ood_dataset['validation'])

id_tr_df['pred_labels'] =  id_tr_pred_labels
id_te_df['pred_labels'] =  id_te_pred_labels
ood_tr_df['pred_labels'] = ood_tr_pred_labels
ood_te_df['pred_labels'] =  ood_te_pred_labels

# Metrics
print('In domain train accuracy: ', id_tr_df[['pred_labels', 'labels']].apply(lambda x: len([i for i in x[1].split(', ') if i in x[0].split(', ')]) == len(x[1].split(', ')), axis=1).sum()/len(id_tr_df))
print('In domain test accuracy: ', id_te_df[['pred_labels', 'labels']].apply(lambda x: len([i for i in x[1].split(', ') if i in x[0].split(', ')]) == len(x[1].split(', ')), axis=1).sum()/len(id_te_df))

ood_df = pd.concat([ood_tr_df, ood_te_df])
print('Out of domain accuracy: ', ood_te_df[['pred_labels', 'labels']].apply(lambda x: len([i for i in x[1].split(', ') if i in x[0].split(', ')]) == len(x[1].split(', ')), axis=1).sum()/len(ood_te_df))

#Dump outputs
dump_path = '/'.join(model_out_path.split('/')[:-1])
id_tr_filename = t5_exp.get_csv_filename(experiment_id)[0]
id_tr_df.to_csv(os.path.join(dump_path, id_tr_filename), index = False)

id_te_filename = t5_exp.get_csv_filename(experiment_id)[1]
id_te_df.to_csv(os.path.join(dump_path, id_te_filename), index = False)

ood_tr_filename = t5_exp.get_csv_filename(experiment_id)[2]
ood_tr_df.to_csv(os.path.join(dump_path, ood_tr_filename), index = False)

ood_te_filename = t5_exp.get_csv_filename(experiment_id)[3]
ood_te_df.to_csv(os.path.join(dump_path, ood_te_filename), index = False)

In domain train accuracy:  0.8827586206896552
In domain test accuracy:  0.84625
Out of domain accuracy:  0.54375


In [ ]:
id_trdf = pd.read_csv('./T5/ATE/t5-base-laptops_vanilla/laptops_train.csv')
id_tedf = pd.read_csv('./T5/ATE/t5-base-laptops_vanilla/laptops_test.csv')
ood_tedf = pd.read_csv('./T5/ATE/t5-base-laptops_vanilla/restaurants_test.csv')

def get_f1_acc(dataframe, true_col, pred_col):
    total_pred = 0
    total_gt = 0
    tp = 0
    for gt, pred in zip(dataframe[true_col][:], dataframe[pred_col][:]):
        gt_list = gt.split(', ')
        pred_list = pred.split(', ')
        total_pred+=len(pred_list)
        total_gt+=len(gt_list)
        for gt_val in gt_list:
            for pred_val in pred_list:
                if pred_val in gt_val:
                    tp+=1
    p = tp/total_pred
    r = tp/total_gt
    return p, r, 2*p*r/(p+r)

print('Train precision:', get_f1_acc(id_trdf, 'labels', 'pred_labels')[0])
print('Test precision:', get_f1_acc(id_tedf, 'labels', 'pred_labels')[0])
print('OOD Test precision:', get_f1_acc(ood_tedf, 'labels', 'pred_labels')[0])

print('Train recall:', get_f1_acc(id_trdf, 'labels', 'pred_labels')[1])
print('Test recall:', get_f1_acc(id_tedf, 'labels', 'pred_labels')[1])
print('OOD Test recall:', get_f1_acc(ood_tedf, 'labels', 'pred_labels')[1])

print('Train F1:', get_f1_acc(id_trdf, 'labels', 'pred_labels')[2])
print('Test F1:', get_f1_acc(id_tedf, 'labels', 'pred_labels')[2])
print('OOD Test F1:', get_f1_acc(ood_tedf, 'labels', 'pred_labels')[2])

Train precision: 0.9493340297727866
Test precision: 0.9317023445463812
OOD Test precision: 0.8194044188280499
Train recall: 0.9284802043422733
Test recall: 0.8856589147286822
OOD Test recall: 0.6408715251690458
Train F1: 0.9387913223140495
Test F1: 0.9080973671137605
OOD Test F1: 0.7192242833052276
